# Sentinel-2 Timelapse (2017-2024)

Timelapse video for the AOI using Sentinel-2 SR, one frame per year (frames per second = 2). Edit dates/paths as needed.

In [13]:
import ee
ee.Initialize()

# -----------------------------
# AOI
# -----------------------------
coords = [
    [140.02059380558322,36.05180094427441],
    [140.03981987980197,36.05180094427441],
    [140.03981987980197,36.0648457180497],
    [140.02059380558322,36.0648457180497],
    [140.02059380558322,36.05180094427441]
]
aoi = ee.Geometry.Polygon([coords])

# -----------------------------
# Yearly composites
# -----------------------------


def yearly_ic(collection, start, end):
    years = ee.List.sequence(start, end)

    def per_year(y):
        y = ee.Number(y)
        imgs = collection.filter(ee.Filter.calendarRange(y, y, "year"))
        median = imgs.median().clip(aoi).set("year", y)
        return median

    return ee.ImageCollection.fromImages(years.map(per_year))


# -----------------------------
# Sentinel-2
# -----------------------------
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(aoi)
    .filterDate("2017-01-01", "2024-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
)

yearly = yearly_ic(collection, 2017, 2024)

# -----------------------------
# Export each image properly
# -----------------------------
years = list(range(2017, 2025))

for i, y in enumerate(years):
    img = ee.Image(yearly.filter(ee.Filter.eq("year", y)).first())

    # Visualization
    vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}
    vis_img = img.visualize(**vis)

    # ---- Custom Filename ----
    filename = f"{y}_agriculture"
    task = ee.batch.Export.image.toDrive(
        image=vis_img,
        description=filename,
        folder="yearly_frames",
        fileNamePrefix=f"{y}",
        scale=10,
        region=aoi,
        crs="EPSG:4326"
    )
    task.start()
    print("Export started for:", y)

Export started for: 2017
Export started for: 2018
Export started for: 2019
Export started for: 2020
Export started for: 2021
Export started for: 2022
Export started for: 2023
Export started for: 2024


In [14]:
from PIL import Image, ImageDraw, ImageFont
import os

input_dir = "frames"
output_dir = "annotated_frames_big"
os.makedirs(output_dir, exist_ok=True)

# AOI bounds
lat_min = 36.05180094427441
lat_max = 36.0648457180497
lon_min = 140.02059380558322
lon_max = 140.03981987980197

lat_grid = (lat_min + lat_max)/2
lon_grid = (lon_min + lon_max)/2

# upscale size
TARGET_W = 1000
TARGET_H = 1000

try:
    font = ImageFont.truetype("arial.ttf", 40)
except:
    font = ImageFont.load_default()


def annotate_big(input_path, output_path, year):
    img = Image.open(input_path).convert("RGB")
    img = img.resize((TARGET_W, TARGET_H), Image.NEAREST)

    draw = ImageDraw.Draw(img)

    # grid positions
    y_grid = int(TARGET_H * (1 - (lat_grid - lat_min)/(lat_max - lat_min)))
    x_grid = int(TARGET_W * ((lon_grid - lon_min)/(lon_max - lon_min)))

    # draw grid
    draw.line([(0, y_grid), (TARGET_W, y_grid)], fill="yellow", width=4)
    draw.line([(x_grid, 0), (x_grid, TARGET_H)], fill="yellow", width=4)

    # lat/lon labels
    draw.text((10, y_grid - 35), f"{lat_grid:.5f}", fill="white", font=font)
    draw.text((x_grid + 15, 10), f"{lon_grid:.5f}", fill="white", font=font)

    # ---------- YEAR LABEL (TOP-RIGHT, BIG FONT) ----------
    try:
        year_font = ImageFont.truetype("arial.ttf", 70)
    except:
        year_font = ImageFont.load_default()

    # use PIL >=10 bounding box
    bbox = draw.textbbox((0, 0), year, font=year_font)
    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]

    padding = 20

    draw.text(
        (TARGET_W - text_w - padding, padding),
        year,
        fill="white",
        font=year_font
    )
    # -------------------------------------------------------

    img.save(output_path)




# process all frames
for fname in sorted(os.listdir(input_dir)):
    if fname.endswith(".tif") or fname.endswith(".png"):
        year = os.path.splitext(fname)[0]
        annotate_big(
            os.path.join(input_dir, fname),
            os.path.join(output_dir, f"{year}.png"),
            year
        )
        print("Annotated:", year)

print("Saved big annotated frames in:", output_dir)

Annotated: 2017
Annotated: 2018
Annotated: 2019
Annotated: 2020
Annotated: 2021
Annotated: 2022
Annotated: 2023
Annotated: 2024
Saved big annotated frames in: annotated_frames_big


In [15]:
import imageio.v2 as imageio
import os

frames = []
for f in sorted(os.listdir("annotated_frames_big")):
    if f.endswith(".png"):
        frames.append(imageio.imread(os.path.join("annotated_frames_big", f)))

imageio.mimsave("timelapse_agriculture.gif", frames, fps=1)
print("GIF saved as timelapse_big.gif")

GIF saved as timelapse_big.gif
